In [1]:
include("./bayes.jl")
include("./helpermodules.jl")
using Gtk, Gtk.ShortNames ,Word2Vec, LIBSVM, JLD, SVR, .Bayes, .Sentiment140DataFrame, .TweetFormat, DataFrames, CSV

### Load ML Model

In [2]:
model = loadnb("naivebayes")

NaiveBayes(8604079, 9374817, Dict{AbstractString,Int64}("beyb" => 1,"rearrangement" => 2,"frogger" => 2,"wtccs" => 2,"aunie" => 1,"twighlight" => 5,"yallah" => 1,"grp" => 6,"montery" => 1,"swimms" => 1…), Dict{AbstractString,Int64}("beyb" => 3,"canclled" => 2,"frogger" => 3,"collegealready" => 1,"twighlight" => 3,"tunnell" => 1,"mysqls" => 1,"grp" => 13,"yallah" => 1,"snowborder" => 1…))

### The Following is the main menu

In [3]:
mainWin = GtkWindow("Home", 400, 160,visible = false)
vboxMain = GtkBox(:v)
hboxMain1 = GtkBox(:h)
sentimentID = GtkButton("\n\n    Sentiment Identifier      \n\n")
screen = Gtk.GAccessor.style_context(sentimentID)
similarPhrase = GtkButton("\n\n     Find a similar phrase    \n\n")
mainMenuLabel = GtkLabel("Main Menu")
push!(hboxMain1,sentimentID)
push!(hboxMain1,similarPhrase)
push!(vboxMain, mainMenuLabel)
push!(vboxMain, hboxMain1)
push!(mainWin, vboxMain)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Home", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=160, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE

### The Following is for the sentiment analysis feature

In [4]:
sentWin = GtkWindow("Sentiment Identifier", 475,150,visible = false)
prompt = GtkLabel("")
GAccessor.markup(prompt,"""<b>Please enter in a sentence or phrase that you wish to get the sentiment for below.</b>\n""")
#User entry
ent = GtkEntry()
set_gtk_property!(ent,:text, "Enter text here... ")

# set_gtk_property!(ent,:default-width, 400)
enterButton = GtkButton("Enter")
hbox = GtkBox(:h)
push!(hbox, ent)
push!(hbox, enterButton)

label = GtkLabel("Your text contains a ... sentiment.")
vbox = GtkBox(:v)
homeButton = GtkButton("Home")
push!(vbox,prompt)
push!(vbox, hbox)
push!(vbox, label)
push!(vbox, homeButton)
push!(sentWin, vbox)
function on_enter_click(w)
    str = get_gtk_property(ent,:text,String)
    if predict_positive(str, model) == true
        GAccessor.text(label,"Your text contains a positive sentiment.")
    else
        GAccessor.text(label,"Your text contains a negative sentiment.")
    end
end
signal_connect(on_enter_click, enterButton,"clicked")

0x0000008d

### The Following is for similar phrase feature

In [5]:
word2vec_model = wordvectors("tweets-vec.txt")
df = getdf("train.csv")
function cosine(vectors, tweet, n=10)
    metrics = vectors'*tweet
    topn_positions = sortperm(metrics[:], rev = true)[1:n]
    topn_metrics = metrics[topn_positions]
    return topn_positions, topn_metrics
end
function cosine_similar_tweets(vectors, tweets, tweet, n=10)
    index, metr = cosine(vectors, tweet, n)
    return tweets[index]
end
mag(x) = sqrt(sum(x.^2))
function createFeatureVector(sentence)
    wordCount = 0
    vector = zeros(size(word2vec_model)[1])
    sentence = split(sentence)
    for word in sentence
        try
            vector =  vector + get_vector(word2vec_model, word)
            wordCount = wordCount + 1
        catch
            nothing
        end
    end
    if wordCount > 0
        vector = vector / mag(vector)
    end
    return vector
end
denoised_tweets = removenoise.(df.tweet)
features = hcat(createFeatureVector.(denoised_tweets)...) # hcat converts Array{Array{Float64,1},1}

100×1599999 Array{Float64,2}:
 -0.0325923    0.121593     0.09821      …   0.0540014     0.0344834
  0.0304493   -0.106964    -0.177016        -0.00233518    0.0629805
  0.0677769    0.127549     0.200062         0.100629      0.0945113
  0.175488     0.0223481    0.0436894        0.0190467     0.0229244
 -0.0457897   -0.0506411   -0.0668149       -0.00238385   -0.0552718
  0.144236    -0.00592271   0.000870906  …   0.134908      0.0868254
 -0.102736    -0.124215     0.0640128       -0.00747444   -0.0226328
 -0.0637817   -0.0759408    0.0631953       -0.0837179     0.101359
  0.0512155    0.0024268    0.057402        -0.0821082     0.0388765
 -0.299816    -0.391158    -0.31233         -0.0470532    -0.0577424
 -0.0766045    0.00858665  -0.0999754    …   0.0834366    -0.245841
  0.0196364   -0.0343909    0.0854022        0.398008      0.216997
 -0.00114507   0.00282725  -0.16601         -0.235859     -0.10484
  ⋮                                      ⋱                
  0.0524998    0.06

In [6]:
simPhraseWin = GtkWindow("Similar Phrase", 400, 220,visible = false)
prompt2 = GtkLabel("")
GAccessor.markup(prompt,"""<b>Please enter in a sentence or phrase that you wish to find similar examples of.</b>\n""")
#User entry
ent2 = GtkEntry()
set_gtk_property!(ent2,:text, "Enter text here... ")

# set_gtk_property!(ent,:default-width, 400)
enterButton2 = GtkButton("Enter")
hbox2 = GtkBox(:h)
push!(hbox2, ent2)
push!(hbox2, enterButton2)

label2 = GtkLabel("")
vbox2 = GtkBox(:v)
homeButton2 = GtkButton("Home")
push!(vbox2,prompt2)
push!(vbox2, hbox2)
push!(vbox2, label2)
push!(vbox2, homeButton2)
push!(simPhraseWin, vbox2)
function on_enter_click2(w)
    str = get_gtk_property(ent2,:text,String)
    tweet_vector = createFeatureVector(removenoise(str))
    similar_tweets = cosine_similar_tweets(features, df.tweet, tweet_vector, 5)
    similar_tweet_label = join(similar_tweets, "\n")
    GAccessor.text(label2,string(similar_tweet_label))
#     ls = GtkListStore(String, Int, Bool)
#     for tweet in similar_tweets
#         push!(ls,tweet)
#     end
#     rTxt = GtkCellRendererText()
#     c1 = GtkTreeViewColumn("Tweet", rTxt, Dict([("text",0)]))
#     push!(tv, c1)
#     pop!(simPhraseWin, vbox2)
#     push!(vbox2, tv)
end
signal_connect(on_enter_click2, enterButton2,"clicked")

0x000000da

### Functions for changing windows

In [7]:
function on_sentID_click(w)
    visible(mainWin, false)
    Gtk.showall(sentWin)
end
function on_simPhrase_click(w)
    visible(mainWin, false)
    Gtk.showall(simPhraseWin)
end
function on_home_click(w)
    visible(sentWin, false)
    Gtk.showall(mainWin)
end   
function on_home_click2(w)
    visible(simPhraseWin, false)
    Gtk.showall(mainWin)
end   
signal_connect(on_home_click2, homeButton2, "clicked")


signal_connect(on_home_click, homeButton, "clicked")
signal_connect(on_sentID_click, sentimentID,"clicked")
signal_connect(on_simPhrase_click, similarPhrase,"clicked")

0x000000e2

In [8]:
Gtk.showall(mainWin)  


GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Home", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=160, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE,